<a href="https://colab.research.google.com/github/Janhavi0077/HPC-and-DL/blob/main/ParallelMergeSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version
%env OMP_NUM_THREADS=3

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
env: OMP_NUM_THREADS=3


In [2]:
%%writefile merge.cpp

#include<iostream>
#include<omp.h>

using namespace std;

void mergesort(int a[], int i, int j);
void merge(int a[], int i1, int j1, int i2, int j2);

void mergesort(int a[], int i, int j)
{
    if (i < j)
    {
        int mid = (i + j) / 2;

        {

            {
                mergesort(a, i, mid);
            }

            {
                mergesort(a, mid + 1, j);
            }
        }
        merge(a, i, mid, mid + 1, j);
    }
}

void paramergesort(int a[], int i, int j)
{
    if (i < j)
    {
        int mid = (i + j) / 2;
#pragma omp parallel sections
        {
#pragma omp section
            {
                mergesort(a, i, mid);
            }
#pragma omp section
            {
                mergesort(a, mid + 1, j);
            }
        }
        merge(a, i, mid, mid + 1, j);
    }
}


void merge(int a[], int i1, int j1, int i2, int j2)
{
    int temp[100000];
    int i, j, k;
    i = i1;
    j = i2;
    k = 0;

    while (i <= j1 && j <= j2)
    {
        if (a[i] < a[j])
        {
            temp[k++] = a[i++];
        }
        else
        {
            temp[k++] = a[j++];
        }
    }

    while (i <= j1)
    {
        temp[k++] = a[i++];
    }

    while (j <= j2)
    {
        temp[k++] = a[j++];
    }

    for (i = i1, j = 0; i <= j2; i++, j++)
    {
        a[i] = temp[j];
    }
}

int main()
{
    int *a, n;

    // Merge Sort
    cout << "\nEnter total number of elements for merge sort: ";
    cin >> n;
    a = new int[n];
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 100; // Generate random numbers between 0 and 99
    }

    double start_time = omp_get_wtime(); // start timer for sequential merge sort
    mergesort(a, 0, n - 1);
    double end_time = omp_get_wtime(); // end timer for sequential merge sort

    cout << "\nSorted array after sequential merge sort:\n";
    for (int i = 0; i < n; i++)
    {
        cout << a[i] << " ";
    }
    cout << "\nTime taken by sequential merge sort: " << end_time - start_time << " seconds" << endl;

    start_time = omp_get_wtime(); // start timer for parallel merge sort
    paramergesort(a, 0, n - 1);

    end_time = omp_get_wtime(); // end timer for parallel merge sort

    cout << "\nSorted array after parallel merge sort:\n";
    for (int i = 0; i < n; i++)
    {
        cout << a[i] << " ";
    }
    cout << "\nTime taken by parallel merge sort: " << end_time - start_time << " seconds" << endl;

    delete[] a; // Don't forget to free the allocated memory

    return 0;
}

Writing merge.cpp


In [3]:
!g++ -fopenmp merge.cpp -o merge

In [4]:
!./merge


Enter total number of elements for merge sort: 7

Sorted array after sequential merge sort:
15 35 77 83 86 86 93 
Time taken by sequential merge sort: 0.000335839 seconds

Sorted array after parallel merge sort:
15 35 77 83 86 86 93 
Time taken by parallel merge sort: 0.000181336 seconds
